In [34]:
from SimPy.Simulation import *
import random
import numpy as np
import math
import read_data as rd
import draw_emp as de

In [35]:
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper

In [36]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = de.draw_empirical(AllArrivals,random.random())
            yield hold, self, t

In [37]:
class Arrival(Process):
    """an arrival"""
    n=0 #class variable, number in system

    def run(self):
        Arrival.n +=1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        yield request, self, G.server
        yield hold, self, de.draw_empirical(AllServices,random.random())
        yield release, self, G.server
        Arrival.n -=1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)


In [38]:
class G:
    server = 'dummy'
    delaymon = 'Monitor' #observe time spent in system
    numbermon = "Monitor" #observe number customer in system

In [46]:
def model(N, maxtime, rvseed,K):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(K)
    G.delaymon = Monitor()
    G.numbermon = Monitor()


    Arrival.n = 0
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    return W, L

In [47]:
filenames = ["Data Collection\KevinYeData.txt",
             "Data Collection\PatrickQData.txt",
             "Data Collection\TamaHoareData.txt",
             "Data Collection\dataviviandong.txt"]
results = rd.ReadData(filenames=filenames)
AllArrivals = results[0]
AllServices = results[1]

The LAB Cafe is open from 7.30am to 3pm a total of 7.5 hours per day which is equivalent to 27000 seconds which would be the maxtime of each simulation.

In [49]:
## Experiment ----------------

for Kap in [3]:
    allW = []
    allL = []
    for k in range(50):
        seed = 123*k
        result = model(N=10000, maxtime=27000, rvseed=seed,K=Kap)
        allW.append(result[0])
        allL.append(result[1])

    print("=================")
    print("When K=",Kap)
    print("    Estimate of W:", np.mean(allW))
    print("    Conf int of W:", conf(allW))
    print("    Estimate of L:", np.mean(allL))
    print("    Conf int of L:", conf(allL))

When K= 3
    Estimate of W: 127.14403384042575
    Conf int of W: (125.07850823449746, 129.20955944635404)
    Estimate of L: 1.0853339772074488
    Conf int of L: (1.053049533747219, 1.1176184206676787)
